In [6]:
import pandas as pd
import numpy as np
import gurobipy as gp
import os
import time
import datetime


### 读取容量情景数据

In [7]:
zdep=r'../data with adjustment/0.scene generation model/DEP'
zarr=r'../data with adjustment/0.scene generation model/ARR'

L=[[],[]]#L[0]为data_dep，L[1]为data_arr，存储初始的210个dep、arr容量数据
#导入def的输入
def read_data(zdep,zarr,L):
    #得到地址zdep和zarr下的csv文件列表[0:3]
    list_dep = os.listdir(zdep)[122:150]
    list_arr = os.listdir(zarr)[122:150]
    ADD=[list_dep,list_arr]
    fumulu=[zdep,zarr]
    #将csv文件列表中的csv文件读取为dataframe
    for j,i,h in zip(L,ADD,fumulu):#j为data_dep或data_arr，i为list_dep或list_arr,j和i一一对应,即data_dep对应list_dep,data_arr对应list_arr
        for k in range(len(i)):
            j.append(pd.read_csv(os.path.join(h, i[k])).iloc[:, 1:])
    return L
C=read_data(zdep,zarr,L)


### 容量调整

In [8]:
A=['ZGGG', 'ZSPD', 'ZGSZ', 'ZBAA', 'ZUTF', 'ZPPP', 'ZLXY', 'ZUCK', 'ZSHC', 'ZBAD', 'ZSSS', 'ZSNJ', 'ZUUU', 'ZHHH', 'ZGHA', 'ZHCC', 'ZSAM', 'ZWWW', 'ZSQD', 'ZJHK', 'ZYTX', 'ZBTJ', 'ZYHB', 'ZUGY', 'ZSJN', 'ZYTL', 'ZJSY', 'ZLLL', 'ZYCC', 'ZGNN', 'ZBHH', 'ZBYN', 'ZSNB', 'ZSOF', 'ZSFZ', 'ZSWZ', 'ZSCN', 'ZGSD', 'ZSWX', 'ZBSJ', 'ZSYT', 'ZLXN', 'ZLIC', 'ZPLJ', 'ZGOW', 'ZPJH', 'ZSQZ', 'ZGKL', 'ZULS', 'ZSCG', 'ZSNT', 'ZHYC', 'ZBDS', 'ZSYA', 'ZUMY', 'ZSWH', 'ZSZS', 'ZSXZ', 'ZBLA', 'ZWSH', 'ZWKL', 'ZLYL', 'ZPDL', 'ZULZ', 'ZSYW', 'ZGZJ', 'ZGHZ', 'ZSLY', 'ZBYC', 'ZHXF', 'ZWAK', 'ZBOW', 'ZPMS', 'ZWYN', 'ZUYB', 'ZBCF', 'ZSLQ', 'ZUMT', 'ZSGZ', 'ZGBH', 'ZBTL', 'ZSSH', 'ZHLY', 'ZSYN', 'ZUZY', 'ZGDY', 'ZSLG', 'ZSWA', 'ZHES', 'ZWTN', 'ZHNY', 'ZBXH', 'ZGCD', 'ZHSY', 'ZUDA', 'ZBUL', 'ZLDH', 'ZSRZ', 'ZUXC', 'ZSHZ', 'ZGZH', 'ZSJU', 'ZYYJ', 'ZSGS', 'ZBLF', 'ZSJG', 'ZSDY', 'ZUWX', 'ZSWF', 'ZUTR', 'ZZZZ', 'ZSYC', 'ZUYI', 'ZLYA', 'ZGHY', 'ZHJZ', 'ZYJM', 'ZUNC', 'ZUNZ', 'ZGFS', 'ZYDQ', 'ZSTX', 'ZGCJ', 'ZSAQ', 'ZSFY', 'ZBSN', 'ZMCK', 'ZBXZ', 'ZBLL', 'ZHEC', 'ZGYY', 'ZBUH', 'ZWKC', 'ZBDT', 'ZBHD', 'ZPBS', 'ZJQH', 'ZYMD', 'ZSJD', 'ZWKM', 'ZUGU', 'ZLQY', 'ZBCZ', 'ZPDQ', 'ZWTS', 'ZPTC', 'ZLLN', 'ZUBZ', 'ZBYZ', 'ZGCZ', 'ZGWZ', 'ZSSR', 'ZWAL', 'ZWAT', 'ZHXY', 'ZWSC', 'ZWBL', 'ZUBJ', 'ZUBD', 'ZPZT', 'ZBZJ', 'ZWHZ', 'ZLZY', 'ZBMZ', 'ZWHM', 'ZLHZ', 'ZBER', 'ZLAK', 'ZWTC', 'ZGYL', 'ZLYS', 'ZSJH', 'ZBZL', 'ZUJZ', 'ZYBS', 'ZYQQ', 'ZGSG', 'ZLGM', 'ZSJJ', 'ZWTL', 'ZLGY', 'ZYJX', 'ZGMX', 'ZSSM', 'ZSWY', 'ZUQJ', 'ZBHZ', 'ZBCD', 'ZPWS', 'ZSLO', 'ZPJM', 'ZYHE', 'ZGBS', 'ZUGH', 'ZUDC', 'ZLJC', 'ZBAL', 'ZYSQ', 'ZWYT', 'ZYDD', 'ZPLC', 'ZYBA', 'ZBDH', 'ZPCW', 'ZYJZ', 'ZYTN', 'ZUKJ', 'ZYJD', 'ZUAL', 'ZYCY', 'ZWCM', 'ZLZW', 'ZYYK', 'ZWRQ', 'ZYLD', 'ZUZH', 'ZYJS', 'ZYMH', 'ZURK', 'ZGSY', 'ZPSM', 'ZLGL', 'ZYFY', 'ZGLG', 'ZLJQ', 'ZGHC', 'ZUNP', 'ZLHX', 'ZWNL', 'ZLDL', 'ZBUT', 'ZUKD', 'ZBES', 'ZUAS', 'ZUGZ', 'ZWFY', 'ZUHY', 'ZWKN', 'ZBEN', 'ZYDU', 'ZUWS', 'ZYCH', 'ZPNL', 'ZBAR', 'ZJYX', 'ZBUC', 'ZUPS', 'ZUWL', 'ZWTK', 'ZWZS', 'ZHSN', 'ZLHB', 'ZYNJ', 'ZUSH', 'ZUDR', 'ZMUB', 'ZLTS', 'ZLYN', 'ZGUH', 'ZKPY', 'ZSPL', 'ZBXJ', 'ZUXJ', 'ZGXX', 'ZUSN', 'ZHJM', 'ZLPC', 'ZGPT', 'ZGNT', 'ZGYJ', 'ZBXT', 'ZYBC', 'ZHQQ', 'ZBYL', 'ZGLD', 'ZHAY', 'ZGLS', 'ZLXH', 'ZMKD', 'ZYAS', 'ZBSG', 'ZBXR', 'ZHDY']
dict_air_index={}
for i in range(len(A)):
    dict_air_index[A[i]]=i
print(dict_air_index)


{'ZGGG': 0, 'ZSPD': 1, 'ZGSZ': 2, 'ZBAA': 3, 'ZUTF': 4, 'ZPPP': 5, 'ZLXY': 6, 'ZUCK': 7, 'ZSHC': 8, 'ZBAD': 9, 'ZSSS': 10, 'ZSNJ': 11, 'ZUUU': 12, 'ZHHH': 13, 'ZGHA': 14, 'ZHCC': 15, 'ZSAM': 16, 'ZWWW': 17, 'ZSQD': 18, 'ZJHK': 19, 'ZYTX': 20, 'ZBTJ': 21, 'ZYHB': 22, 'ZUGY': 23, 'ZSJN': 24, 'ZYTL': 25, 'ZJSY': 26, 'ZLLL': 27, 'ZYCC': 28, 'ZGNN': 29, 'ZBHH': 30, 'ZBYN': 31, 'ZSNB': 32, 'ZSOF': 33, 'ZSFZ': 34, 'ZSWZ': 35, 'ZSCN': 36, 'ZGSD': 37, 'ZSWX': 38, 'ZBSJ': 39, 'ZSYT': 40, 'ZLXN': 41, 'ZLIC': 42, 'ZPLJ': 43, 'ZGOW': 44, 'ZPJH': 45, 'ZSQZ': 46, 'ZGKL': 47, 'ZULS': 48, 'ZSCG': 49, 'ZSNT': 50, 'ZHYC': 51, 'ZBDS': 52, 'ZSYA': 53, 'ZUMY': 54, 'ZSWH': 55, 'ZSZS': 56, 'ZSXZ': 57, 'ZBLA': 58, 'ZWSH': 59, 'ZWKL': 60, 'ZLYL': 61, 'ZPDL': 62, 'ZULZ': 63, 'ZSYW': 64, 'ZGZJ': 65, 'ZGHZ': 66, 'ZSLY': 67, 'ZBYC': 68, 'ZHXF': 69, 'ZWAK': 70, 'ZBOW': 71, 'ZPMS': 72, 'ZWYN': 73, 'ZUYB': 74, 'ZBCF': 75, 'ZSLQ': 76, 'ZUMT': 77, 'ZSGZ': 78, 'ZGBH': 79, 'ZBTL': 80, 'ZSSH': 81, 'ZHLY': 82, 'ZSYN': 83, '

### 设置模型需要的集合

In [9]:
#设置情景集合，也就是天数
scene=[i for i in range(0,28)]


### 设置模型需要的参数

In [10]:
#计算28天各天情景的曼哈顿距离，避免异常值的影响
def distance(C):
    data_dep = np.array(C[0])
    data_arr = np.array(C[1])
    n = len(data_dep)
    m = len(data_arr)
    D_dep = np.zeros((n, m))
    D_arr = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            D_dep[i, j] = np.sum(np.abs(data_dep[i] - data_dep[j]))
            D_arr[i, j] = np.sum(np.abs(data_arr[i] - data_arr[j]))
    D=D_dep+D_arr
    return D

dis=distance(C)
dis=pd.DataFrame(dis)
print(dis)#Excel检验了一下，结果正确


         0       1        2        3       4       5       6       7       8   \
0       0.0  8946.0   8919.0   7674.0  8439.0  7070.0  8772.0  7118.0  9152.0   
1    8946.0     0.0   8139.0   9158.0  7483.0  8774.0  7548.0  9020.0  7268.0   
2    8919.0  8139.0      0.0   9513.0  7342.0  9079.0  7903.0  9185.0  8411.0   
3    7674.0  9158.0   9513.0      0.0  8659.0  7402.0  8896.0  7690.0  9330.0   
4    8439.0  7483.0   7342.0   8659.0     0.0  8121.0  6133.0  8331.0  7075.0   
5    7070.0  8774.0   9079.0   7402.0  8121.0     0.0  8176.0  6600.0  8692.0   
6    8772.0  7548.0   7903.0   8896.0  6133.0  8176.0     0.0  8148.0  6716.0   
7    7118.0  9020.0   9185.0   7690.0  8331.0  6600.0  8148.0     0.0  8116.0   
8    9152.0  7268.0   8411.0   9330.0  7075.0  8692.0  6716.0  8116.0     0.0   
9    8943.0  7903.0   7514.0   9411.0  7056.0  8575.0  6517.0  8301.0  6863.0   
10   7463.0  9149.0   9414.0   7485.0  8316.0  6801.0  8061.0  6707.0  8549.0   
11   9382.0  8616.0   8577.0

In [17]:
#设置生成情景个数的参数n
n=3
#设置每个情景簇至少包含的情景个数的参数N
N=7


### 定义模型

In [ ]:
def qingjingshengcheng(scene,dis,n,N):
    #创建模型
    m=gp.Model()
    #添加决策变量、间接变量【这些是待定值，未知数】
    #s是否在生成情景S的簇中
    var_x_sS=gp.tuplelist([(s,S) for s in scene for S in scene])
    #s是否在生成的情景S的结果集合中
    var_y_S=gp.tuplelist([(S) for S in scene])
    #指示变量，写目标函数需要
    var_z_sS=gp.tuplelist([(s,S) for s in scene for S in scene])
    x_sS=m.addVars(var_x_sS,lb=0,ub=1,vtype=gp.GRB.BINARY,name='x_sS')
    y_S=m.addVars(var_y_S,lb=0,ub=1,vtype=gp.GRB.BINARY,name='y_S')
    z_sS = m.addVars(var_z_sS, lb=-1, ub=1, vtype=gp.GRB.INTEGER, name='z_sS')
    #添加约束条件
    #约束条件1：限制最终生成的情景数量为n
    m.addConstr((gp.quicksum(y_S[i] for i in scene)==n),name='con1')
    #约束条件2：设置决策变量x_sS和y_s的关系，即x_sS[i,j]<=y_s[j]
    m.addConstrs((x_sS[i,j]<=y_S[j] for i in scene for j in scene),name='con2')
    #约束条件3：一个情景只能属于一个簇
    m.addConstrs((gp.quicksum(x_sS[i,j] for j in scene)==1 for i in scene),name='con3')
    #约束条件4：一个情景簇至少包含N个情景
    m.addConstrs((gp.quicksum(y_S[j]*x_sS[i,j] for i in scene)>=y_S[j]*N for j in scene),name='con4')
    #约束条件5：Z_sS与x_sS的关系由Zf函数决定【注意，gurobi的约束不可以进行if判断和布尔运算，但可以使用外部函数】
    #m.addConstrs((z_sS[i,j]==Zf(x_sS[i,j]) for i in scene for j in scene),name='con5')
    m.addConstrs((z_sS[i,j] >= -1 + 2 * x_sS[i,j] for i in scene for j in scene), name='con5a')
    m.addConstrs((z_sS[i,j] <= 1 - 2 * (1 - x_sS[i,j]) for i in scene for j in scene), name='con5b')
    
    #设置目标函数
    m.setObjective((gp.quicksum(dis.iloc[i,j]*z_sS[i,j]*y_S[j] for j in scene for i in scene)),gp.GRB.MINIMIZE)
    
    m.Params.TimeLimit = 7200
    m.update()
    m.optimize()
    
    #输出的是情景簇的编号,即y_S为1的值
    L=[]
    for i in scene:
        if y_S[i].x==1:
            L.append(i)
    
    #输出L中各个情景簇的情景个数
    LL=[]
    for i in L:
        a=0
        for j in scene:
            if x_sS[j,i].x==1:
                a+=1
        LL.append(a)
        #LL归一化
        PP=[i/sum(LL) for i in LL]
        
    # #将模型决策变量结果输出为csv文件，放在桌面
    m.write('{}.sol'.format(n))
    return L,LL,PP

#约束索引之类的方法
L,LL,PP=qingjingshengcheng(scene,dis,n,N)


Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2381 rows, 1596 columns and 5516 nonzeros
Model fingerprint: 0x5cac3526
Model has 756 quadratic objective terms
Model has 28 quadratic constraints
Variable types: 0 continuous, 1596 integer (812 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [7e+00, 7e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 1540 rows and 784 columns
Presolve time: 0.01s
Presolved: 1597 rows, 1568 columns, 5460 nonzeros
Variable types: 0 continuous, 1568 integer (1568 binary)

Root relaxation: objective -7.259958e+05, 559 iterations, 0

In [26]:
print(L,LL,PP)

[2, 20, 24] [7, 9, 12] [0.25, 0.32142857142857145, 0.42857142857142855]


In [ ]:
# for i,j in zip(L,[0,1,2]):
#     C[0][i].to_csv('情景生成数据-200多个双通道3维张量/JIEGUO/DEP/C{}.csv'.format(j))
#     C[1][i].to_csv('情景生成数据-200多个双通道3维张量/JIEGUO/ARR/C{}.csv'.format(j))
